In [1]:
from polars_ds.extensions import StrExt, NumExt
import polars as pl
import numpy as np 

In [2]:
size = 2048
df = pl.DataFrame({
    "a": np.sin(np.array(range(size))),
    "b": np.sin(np.array(range(size))),
    "c": np.sin(np.array(range(size))),
    "d": np.sin(np.array(range(size))),
    "e": np.sin(np.array(range(size))),
})
npa = df["a"].to_numpy()
npb = df["b"].to_numpy()
npc = df["c"].to_numpy()
npd = df["d"].to_numpy()
npe = df["e"].to_numpy()

In [5]:
%%timeit
np.fft.rfft(npa).real

12.4 µs ± 71.7 ns per loop (mean ± std. dev. of 7 runs, 100,000 loops each)


In [4]:
%%timeit
df.select(
    pl.col("a").num_ext.fft().struct.field("re"),
)

114 µs ± 4.7 µs per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [8]:
%%timeit
df.select(
    pl.col("a").num_ext.fft(),
     pl.col("b").num_ext.fft(),
     pl.col("c").num_ext.fft(),
     pl.col("d").num_ext.fft(),
     pl.col("e").num_ext.fft()
)

216 µs ± 6.39 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)


In [9]:
%%timeit 
_ = np.fft.rfft(npa)
_ = np.fft.rfft(npb)
_ = np.fft.rfft(npc)
_ = np.fft.rfft(npd)
_ = np.fft.rfft(npe)

51.1 µs ± 312 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [10]:
df = pl.DataFrame({
    "dummy": ["a"] * 50_000 + ["b"] * 50_000,
    "a": range(100_000),
    "b": range(-100_000, 0),
    "y": pl.Series(range(100_000, 200_000)) + 0.5
})

df.head()

dummy,a,b,y
str,i64,i64,f64
"""a""",0,-100000,100000.5
"""a""",1,-99999,100001.5
"""a""",2,-99998,100002.5
"""a""",3,-99997,100003.5
"""a""",4,-99996,100004.5


In [12]:
%%timeit
df.select(
    pl.col("y").num_ext.lstsq(pl.col("a"), add_bias=True)
)

738 µs ± 12 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)


In [ ]:
df.group_by("dummy").agg(
    pl.col("y").num_ext.lstsq(pl.col("a"), pl.col("b"), add_bias = True)
)

In [ ]:
df.select(pl.col("y").num_ext.lstsq(pl.col("a"), pl.col("b")))

In [ ]:
df.select(
    pl.col("y").num_ext.lstsq(pl.col("a"), pl.col("b")).over("dummy")
)